# Demo: Generation & extraction of the IPEs for the NWM Modules

Generate & extract relevant IPEs for the NWM modules.

__Pre-requisites:__

- `.env` with your AWS credentials

## Modules/Libraries

In [ ]:
import json
import os
from pathlib import Path

from pyiceberg.catalog import load_catalog

from icefabric.helpers import load_creds, load_pyiceberg_config
from icefabric.modules import (
    get_cfe_parameters,
    get_lasam_parameters,
    get_lstm_parameters,
    get_noahowp_parameters,
    get_sacsma_parameters,
    get_smp_parameters,
    get_snow17_parameters,
    get_topmodel_parameters,
    get_troute_parameters,
    get_ueb_parameters,
)
from icefabric.schemas.hydrofabric import HydrofabricDomains

# Changes the current working dir to be the project root
current_working_dir = Path.cwd()
os.chdir(Path.cwd() / "../../")
print(
    f"Changed current working dir from {current_working_dir} to: {Path.cwd()}. This must run at the project root"
)

# dir is where the .env file is located
load_creds(dir=Path.cwd())

# Loading the local pyiceberg config settings
pyiceberg_config = load_pyiceberg_config(Path.cwd())

## User Arguments

In [ ]:
# Assumes your S3 .env file is set up for the NGWPC Test Account
catalog = load_catalog("glue", **{"type": "glue", "glue.region": "us-east-1"})

# Picking a gauge in CONUS and loading upstream connections
domain = HydrofabricDomains.CONUS
upstream_connections_path = Path(f"data/hydrofabric/{domain.value}_upstream_connections.json")

if not upstream_connections_path.exists():
    raise FileNotFoundError

with open(upstream_connections_path) as f:
    data = json.load(f)
    print(
        f"Loading upstream connections generated on: {data['_metadata']['generated_at']} "
        f"from snapshot id: {data['_metadata']['iceberg']['snapshot_id']}"
    )
    upstream_dict = data["upstream_connections"]

# Pick the gauge identifier
# identifier = 'gages-06710385'
identifier = "gages-11280000"

## TopModel

In [ ]:
topmodel_pydantic_models = get_topmodel_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict
)
topmodel_pydantic_models

## OWP NOAH MODULAR

In [ ]:
noahowp_pydantic_models = get_noahowp_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict
)
noahowp_pydantic_models

## LASAM

In [ ]:
lasam_pydantic_models = get_lasam_parameters(
    catalog=catalog,
    namespace=domain.value,
    identifier=identifier,
    upstream_dict=upstream_dict,
    sft_included=True,
    soil_params_file="vG_default_params_HYDRUS.dat",
)
lasam_pydantic_models

In [ ]:
lasam_pydantic_models = get_lasam_parameters(
    catalog=catalog,
    namespace=domain.value,
    identifier=identifier,
    upstream_dict=upstream_dict,
    sft_included=False,
    soil_params_file="vG_default_params_HYDRUS.dat",
)
lasam_pydantic_models

## LSTM

In [ ]:
lstm_pydantic_models = get_lstm_parameters(
    catalog=catalog,
    namespace=domain.value,
    identifier=identifier,
    upstream_dict=upstream_dict,
)
lstm_pydantic_models

## SMP

In [ ]:
module = "CFE-S"  # , 'CFE-X', 'LASAM', 'TopModel'
smp_pydantic_models = get_smp_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict, module=module
)
smp_pydantic_models

## Snow17

In [ ]:
snow17_pydantic_models = get_snow17_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict, envca=False
)

snow17_pydantic_models

In [ ]:
snow17_pydantic_models = get_snow17_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict, envca=True
)
snow17_pydantic_models

## SAC SMA

In [ ]:
sacsma_pydantic_models = get_sacsma_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict, envca=False
)
sacsma_pydantic_models

In [ ]:
sacsma_pydantic_models = get_sacsma_parameters(
    catalog=catalog, namespace=domain.value, identifier=identifier, upstream_dict=upstream_dict, envca=True
)
sacsma_pydantic_models

## T-Route

In [ ]:
troute_pydantic_models = get_troute_parameters(
    catalog=catalog,
    namespace=domain.value,
    identifier=identifier,
    upstream_dict=upstream_dict,
)
troute_pydantic_models

## UEB

In [ ]:
ueb_pydantic_models = get_ueb_parameters(
    catalog, namespace=domain.value, identifier=identifier, envca=False, upstream_dict=upstream_dict
)

ueb_pydantic_models

## CFE

In [ ]:
cfex_pydantic_models = get_cfe_parameters(
    catalog, namespace=domain.value, identifier=identifier, module="CFE-X", upstream_dict=upstream_dict
)
cfex_pydantic_models

In [ ]:
cfes_pydantic_models = get_cfe_parameters(
    catalog,
    namespace=domain.value,
    identifier=identifier,
    module="CFE-S",
    upstream_dict=upstream_dict,
    sft_included=True,
)
cfes_pydantic_models